In [1]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [2]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('Secret_key')

openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import json

#현재 날씨 챗봇 함수 정의
def get_current_weather(date_time, location, unit='celsius'):
    weather_info = {
        'date_time' : date_time,
        'location' : location,
        'unit' : unit,
        'forecast' : ['sunny', 'windy']
    }
    return json.dumps(weather_info)

#함수 설명
functions = [
    {
        "name" : "get_current_weather",
        "description" : "Get the current weather for a given date and location",
        #parameters >> 함수 호출 시 필요한 모든 입력값들에 대한 명세서를 제공하는 역할 (type, properties, required)
        "parameters" : {
            "type" : "object",
            #properties >> 특정 객체가 가질 수 있는 모든 속성과 해당 속성의 타입 및 설명을 명시 (date_time, location)
            "properties" : {
                "date_time" : {
                    "type" : "string",
                    "description" : "date and time"
                },
                "location" : {
                    "type" : "string",
                    "enum" : ["celsius", "fahrenheit"]
                }
            },
            #required >> 필수 속성들
            "required" : ["date_time", "location"]
                },
        },

]

In [4]:
#함수 정의와 함수 설명 매칭하기 위한 용도
available_functions = {
    'get_current_weather' : get_current_weather
}

In [11]:
def llm(input_text, chat_history):

    if len(chat_history) == 0:
        chat_history.append({"role" : "system", "content" : "Act like a friend who is kind and highly empathetic. Respond to the user's input in a friendly and conversational manner in Korean."})

    chat_history.append({"role" : "user", "content" : input_text})
    # print(chat_history)
    # [{'role': 'system', 'content': "Act like a friend who is kind and highly empathetic. Respond to the user's input in a friendly and conversational manner in Korean."}, {'role': 'user', 'content': '넌 누구니?'}]
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = chat_history,
        functions = functions,
        function_call = "auto"
    )
    response_message = response['choices'][0]['message']
    # print(response_message)

    if response_message.get('function_call'): #함수를 호출
        #함수 이름 가져오기
        function_name = response_message['function_call']['name']
        #실제 부를 수 있는 함수
        function_to_call = available_functions[function_name]
        #함수 인자 가져오기
        function_args = json.loads(response_message['function_call']['arguments'])
        #AI가 어떤 함수를 선택했고, 어떤 인자를 입력했는지 >> chat_history에 추가
        chat_history.append(response_message)
        #실제 함수 호출 >> 결과값
        function_response = function_to_call(**function_args)

        chat_history.append(
            {"role":"function"},
            {"name" : function_name, "content" : function_response}
        )

        second_response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = chat_history,
        )

        output = second_response.choices[0].message.content
    else:
        output = response_message['content']
    chat_history.append({"role" : "assistant", "content" : output})
    return output

In [12]:
def chat_with_user(user_message, chat_history):
    ai_message = llm(user_message, chat_history)
    return ai_message

chat_history = [ ]

while True:
    user_message = input('user: > ')
    if user_message.lower() == 'quit':
        break
    ai_message = chat_with_user(user_message, chat_history)
    print(f'ai: {ai_message}')


user: > 안녕
ai: 안녕하세요! 반가워요 😊 어떻게 도와드릴까요?
user: > 위에 코드중에 function_call은 왜 한거야?
ai: function_call은 채팅 상에서 사용자의 요청을 처리하고 함수를 호출하기 위한 코드입니다. 이를 통해 제가 사용자의 질문에 적합한 답변을 찾아드릴 수 있어요. 궁금한 점이 있거나 더 자세한 설명이 필요하시면 언제든지 말씀해주세요! 😊
user: > quit
